In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from PIL import Image
import numpy
import cv2

In [15]:
class ObjectDetection:
    def __init__(self,score,train_mode = False):
        self.score = score
        self.model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
        self.train_mode = train_mode
        if not train_mode:
            self.model.eval()
        self.transform = transforms.ToTensor()
    def object_Detection(self,img_path):
        img = Image.open(img_path)
        img_transform = self.transform(img)
        with torch.no_grad():
            pred= self.model([img_transform]) 
        bboxes,labels,scores = pred[0]['boxes'],pred[0]['labels'],pred[0]['scores']
        mask = scores > self.score
        bboxes = bboxes[mask]
        labels = labels[mask]
        scores = scores[mask]
        return bboxes,scores,labels
    def Draw(self,img_path,bboxes):
        img1 = cv2.imread(img_path)
        for box in bboxes:
            x1,y1,x2,y2 = box.numpy().astype(int)
            cv2.rectangle(img1,(x1,y1),(x2,y2),(255,0,0),1)
        return img1
    def display(self,img1):
        cv2.imshow("Detected Images",img1)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    def process_image(self, image_path):
        if self.train_mode:
                loss = self.object_Detection(image_path)
                print(loss)
        else:
            bboxes, labels, scores = self.object_Detection(image_path)
            image_with_boxes = self.Draw(image_path, bboxes)
            self.display(image_with_boxes)



In [ ]:
if __name__ == "__main__":
    detector = ObjectDetection(score=0.8)#It can be your wish
    detector.process_image("your_image_path")